In [ ]:
import os

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import factorial

import client as cp
import middleware as mp

# Matplot lib settings
%matplotlib notebook
matplotlib.rcParams.update({
        'font.size': 11,
        'font.family': 'sans-serif',
        'xtick.labelsize': 10,
        'ytick.labelsize': 10
    })

# Numpy settings
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [ ]:
def predict(lam, mu, m):
    rho = lam / (m * mu)
    print('lambda', lam)
    print('mu', mu)
    print('m', m)
    print('rho', rho)

    p_0 = 1
    p_0 += (np.power(m * rho, m) / (factorial(m, exact=True) * (1 - rho)))
    for n in range(1, m):
        p_0 += (np.power(m * rho, n) / factorial(n, exact=True))
    p_0 = 1.0 / p_0
#     print('p_0', p_0)
    
    p_q = np.power(m * rho, m) / (factorial(m, exact=True) * (1 - rho)) * p_0
#     print('p_q', p_q)
#     print()

    E_r = 1 / mu * (1 + (p_q / (m * (1 - rho))))

    E_w = p_q / (m * mu * (1 - rho))

    E_n = m * rho + rho * p_q / (1 - rho)

    E_nq = rho * p_q / (1 - rho)
    
    return [E_r * 1000, E_w * 1000, E_n, E_nq]

In [ ]:
a = [predict(lam, 21993 / 5, 5)[0] for lam in range(1, 21993, 2)]
x = np.array(range(1, 21993, 2)) / 21993
fig = plt.figure(figsize=(6, 5))
plt.plot(x, a, label='M/M/m model')
plt.xlabel('Traffic intensity')
plt.ylabel('Response time (ms)')

plt.scatter(bx, by, marker='o', label='Measured', color='red')
plt.scatter(cx, cy, marker='d', label='Adjusted', color='green')

plt.xlim([0, 1.1])
plt.ylim([0, 50])

plt.legend(bbox_to_anchor=(0., 1., 1., 0.), loc=9, ncol=3, 
           mode="expand", borderaxespad=0., fontsize='medium')
plt.tight_layout()
fig.savefig('mmm1.png', dpi=300, bbox_inches='tight', pad_inches=0)

In [ ]:
by = []
bx = []

cy = []
cx = []

base_dir = '/Users/ruifengxu/Development/asl-fall16-project/logs/maximum-throughput/thread-{}-client-{}/'

thread_pool_size = 16

for num_clients in [10, 40, 70, 100, 130]:
    print('============================================')
    print('thread pool size', thread_pool_size)
    print('number clients', num_clients)
    config = {
        'request_type': 'g',
        'start': 16,
        'duration': 30,
        'ci_coefficient': 2.776,
        'num_repetitions': 5,
        'num_client_vms': 3,
        'num_clients_per_vm': num_clients,
        'thread_pool_size': thread_pool_size,
        'replication_factor': 1,
        'base_dir': base_dir.format(thread_pool_size, num_clients)
    } 
    client = cp.aggregate_data(config)
    print('tp mean {:.3f}'.format(client['tp_mean']))
    print('rt mean {:.3f}'.format(client['rt_mean']))
    print()

    lam = client['tp_mean']
    m = 5
    
    config = {
        'request_type': 'g',
        'start': 16,
        'duration': 45,
        'num_repetitions': 5,
        'base_dir': base_dir.format(thread_pool_size, num_clients)
    }
    middleware = mp.aggregate_data(config)
    print('read request {:.3f}'.format(middleware['read_request']))
    print('queue {:.3f}'.format(middleware['queue']))
    print('forward request {:.3f}'.format(middleware['forward_request']))
    print('server {:.3f}'.format(middleware['server']))
    print('send response {:.3f}'.format(middleware['send_response']))
    print('total {:.3f}'.format(middleware['total']))
    print()
        
    total_clients = num_clients * 3
    clients_in_network = (client['rt_mean'] - middleware['total']) * lam / 1000
    clients_in_system = middleware['total'] * lam / 1000
    clients_in_queue = middleware['queue'] * lam / 1000
    clients_in_processing = clients_in_system - clients_in_queue
    scaling_factor = clients_in_processing / m
    print('total clients', total_clients)
    print('clients in network', clients_in_network)
    print('clients in system', clients_in_system)
    print('clients in queue', clients_in_queue)
    print('clients in processing', clients_in_processing)
    print('scaling factor', scaling_factor)
    print()
        
    measured = [
        middleware['total'], 
        middleware['queue'], 
        lam * middleware['total'] / 1000, 
        lam * middleware['queue'] / 1000
    ]        
    print('{:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f}'.format(*measured))
    print()
    
    mu = 21993 / 5
    prediction_maxtp = predict(lam, mu, m)
    print('{:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f}'.format(*prediction_maxtp))
    print('{:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f}'.format(*(np.array(prediction_maxtp) * scaling_factor)))
    print()
    
    by.append(middleware['total'])
    bx.append(lam / 21993)
    cy.append((np.array(prediction_maxtp) * scaling_factor)[0])
    cx.append(lam / 21993)

In [ ]:
replication_factor = 1
write_percentage = 10
thread_pool_size = 16
num_clients_per_vm = 110
base_dir = '/Users/ruifengxu/Development/asl-fall16-project/logs/write/server-{}-replication-{}-write-{}/'

for num_servers in [3, 5, 7]:
    print('============================================')
    print('number servers', num_servers)
    config = {
        'request_type': 'a',
        'start': 16,
        'duration': 30,
        'ci_coefficient': 2.776,
        'num_repetitions': 5,
        'num_client_vms': 3,
        'num_clients_per_vm': num_clients_per_vm,
        'thread_pool_size': thread_pool_size,
        'replication_factor': replication_factor,
        'base_dir': base_dir.format(num_servers, replication_factor, write_percentage)
    } 
    client = cp.aggregate_data(config)
    print('tp mean {:.3f}'.format(client['tp_mean']))
    print('rt mean {:.3f}'.format(client['rt_mean']))
    print()

    config = {
        'request_type': 'g',
        'start': 16,
        'duration': 30,
        'num_repetitions': 5,
        'base_dir': base_dir.format(num_servers, replication_factor, write_percentage),
    }
    middleware_get = mp.aggregate_data(config)
    print('read request {:.3f}'.format(middleware_get['read_request']))
    print('queue {:.3f}'.format(middleware_get['queue']))
    print('forward request {:.3f}'.format(middleware_get['forward_request']))
    print('server {:.3f}'.format(middleware_get['server']))
    print('send response {:.3f}'.format(middleware_get['send_response']))
    print('total {:.3f}'.format(middleware_get['total']))
    print()
    
    args = [16]
    args.append((middleware_get['forward_request'] + middleware_get['server']) / 1000)

    config['request_type'] = 's'
    middleware_set = mp.aggregate_data(config)
    print('read request {:.3f}'.format(middleware_set['read_request']))
    print('queue {:.3f}'.format(middleware_set['queue']))
    print('forward request {:.3f}'.format(middleware_set['forward_request']))
    print('server {:.3f}'.format(middleware_set['server']))
    print('send response {:.3f}'.format(middleware_set['send_response']))
    print('total {:.3f}'.format(middleware_set['total']))
    print()

    args.append((middleware_set['forward_request'] + middleware_set['server']) / 1000)

    w1 = 0.1
    w2 = 0.9
    middleware_read_request = w1 * middleware_set['read_request'] + w2 * middleware_get['read_request']
    middleware_queue = w1 * middleware_set['queue'] + w2 * middleware_get['queue']
    middleware_server = w1 * middleware_set['server'] + w2 * middleware_get['server']
    middleware_total = w1 * middleware_set['total'] + w2 * middleware_get['total']
    
    args.append((client['rt_mean'] - middleware_total) / 1000)
    args.append(middleware_read_request / 1000)


    lam = client['tp_mean']
    m = num_servers
    
#     print('{}, {:.10f}, {:.10f}, {:.10f}, {:.10f}'.format(*args))
    measured = [
        middleware_total,
        middleware_queue,
        lam * middleware_total / 1000, 
        lam * middleware_queue / 1000
    ]        
    print('{:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f}'.format(*measured))
    
    mu = 1 / middleware_server * 1000 * 17
    prediction_maxtp = predict(lam, mu, m)
    print('{:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f}'.format(*prediction_maxtp))
    print('{:<10.3f} {:<10.3f} {:<10.3f} {:<10.3f}'.format(*(np.array(prediction_maxtp) * 17)))
    print()
